# Simple Steganography

Steganography is the practice of concealing a message within another message or a physical object. In computing/electronic contexts, a computer file, message, image, or video is concealed within another file, message, image, or video.

Here, we hide a message inside a RGB image. We write appropriate functions for putting the message into the image and also extracting the message from the image.

In [1]:
from PIL import Image
import numpy as np

### Load the image

In [2]:
image = Image.open("./_MG_3018.jpg")

### Resize the image

We can resize the image if we want to. This is not a necessary step to perform.

Initially we find out the dimensions of the original image.

In [3]:
original_dimensions = (image.width, image.height)
print(f"Dimensions of Original Image = {original_dimensions}")

Dimensions of Original Image = (2667, 4000)


Then we calculate the dimensions of the new image. The maximum structure of the resulting image is defined by the **TARGET_DIM** variable.

For example, <br>
For square images, its size will be **TARGET_DIM x TARGET_DIM** <br>
For landscape images, its width will be equal to **TARGET_DIM** and the height will adjust accordingly <br>
For portrait images, its height will be equal to **TARGET_DIM** and the width will adjust accordingly

In [4]:
# For pictures with equal height and width
TARGET_DIM = 100
if(original_dimensions[0] == original_dimensions[1]):
    factor = original_dimensions[0] // TARGET_DIM
    dimensions = (original_dimensions[0] // factor, original_dimensions[1] // factor)

# For pictures with more width than height
elif(original_dimensions[0] > original_dimensions[1]):
    factor = original_dimensions[0] // TARGET_DIM
    dimensions = (original_dimensions[0] // factor, original_dimensions[1] // factor)

# For pictures with more height than width
elif(original_dimensions[0] < original_dimensions[1]):
    factor = original_dimensions[1] // TARGET_DIM
    dimensions = (original_dimensions[0] // factor, original_dimensions[1] // factor)

print(f"Dimensions of Resized Image = {dimensions}")

Dimensions of Resized Image = (66, 100)


We do not execute the following block because we do not want to resize the image in this case. If we wanted to, we will need to uncomment it.

In [ ]:
# image = image.resize(dimensions)
# image

### Convert Image to Numpy

Image data is essentially a multi-dimensional array. Any modification we intend to perform on it, we have to convert it to an editable array form. For this, we convert it to Numpy arrays.

In [22]:
img_array = np.array(image)

We print the dimensions of the matrix (representing the image). <br>
The first value represents the hight of the image (number of rows in the matrix); the second value represents the width of the image (number of columns in the image); and the third value represents the number of channels in the image. In case of RGB images, we have 3 channels.

In [23]:
print(f"Dimensions of Numpy array = {img_array.shape}")

Dimensions of Numpy array = (4000, 2667, 3)


### Convert ASCII String to Binary String

To store ASCII text, we first convert it to binary because we store it as binary values in the image. We convert each ASCII character to a 8-bit binary value. ASCII can only take up 7 bits because it ranges from 0 to 127. The 8th bit (MSB) is used as a control bit which denotes the end of message.

In [34]:
def stringToBinary(message: str) -> str:
    output = ""
    for ch in message:
        value = (bin(ord(ch)).replace("0b", ""))
        
        padding = 8 - len(value)
        for i in range(padding):
            value = "0" + value
        
        output += value
    return output

### Embed Data into the Image

Embed the binary string into the image provided. The *message* provided to the function should be an ASCII string and the *image* should be a Numpy array representing an RGB image, that is, it should have 3 dimensions.

This function replaces the last bit of each byte with the bits of the binary message. The end of message is represented by a *1* in the MSB of the last byte.

In [27]:
def embedData(message: str, image):
    binary_message = stringToBinary(message)
    
    # Check if the given message will fit into the given image
    assert (len(binary_message) <= (image.shape[0] * image.shape[1])), "Given message is too big to fit in the given image"
    
    index = 0
    done = False
    for channel in range(image.shape[2]):
        for row in range(image.shape[0]):
            for column in range(image.shape[1]):
                if(index == len(binary_message)):
                    image[row][column][channel] = image[row][column][channel] | 1
                    done = True
                    break

                binary = binary_message[index]

                if(binary == "0"):
                    image[row][column][channel] = image[row][column][channel] & 254
                elif(binary == "1"):
                    image[row][column][channel] = image[row][column][channel] | 1

                index += 1

            if(done == True):
                break
        if(done == True):
            break
    
    return image

In [33]:
message = "Steganography is the practice of concealing a message within another message or a physical object. In computing/electronic contexts, a computer file, message, image, or video is concealed within another file, message, image, or video."
embedded_image = embedData(message, img_array)

### Convert Numpy array to Image

In [29]:
new_image = Image.fromarray(embedded_image)

In [ ]:
new_image

### Extract Data from Image

Extract the data which is embedded into the image. The *image* provided should be a Numpy array with 3 dimensions, essentially a RGB image. 

It iterates over all the bytes and extracts the last bits from each byte. It then groups them into 8 bits each and converts it into the corresponding ASCII form. This process continues until a byte is found whose MSB contains a *1* which denotes end of message. It ignores that byte and terminates.

In [31]:
def extractEmbeddedData(image) -> str:
    bit_counter = 1
    bin_data = ""
    message = ""
    for channel in range(image.shape[2]):
        for row in range(image.shape[0]):
            for column in range(image.shape[1]):
                bin_data += str(image[row][column][channel] % 2)

                if(bit_counter % 8 == 0):
                    # Terminating condition
                    if(bin_data[0] == "1"):
                        return message
                    else:
                        message += chr(int(bin_data, 2))

                    # Reset the temporary variables
                    bin_data = ""

                bit_counter += 1

In [35]:
extractEmbeddedData(np.array(new_image))

'Steganography is the practice of concealing a message within another message or a physical object. In computing/electronic contexts, a computer file, message, image, or video is concealed within another file, message, image, or video.'